In [1]:
import tensorflow as tf
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout, Embedding, Concatenate
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.layers import Embedding, Multiply, Reshape
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, LeakyReLU, Conv2DTranspose, Activation, Input
from keras.utils import plot_model
from keras import initializers
from keras.layers import multiply
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets.cifar10 import load_data



# Load data.
There are 50K examples in the training set and 10K in the test set and that each image is a square of 32 by 32 pixels.

In [2]:
(train_x, train_y), (test_x, test_y) = load_data()
print('Train', train_x.shape, train_y.shape)
print('Test', test_x.shape, test_y.shape)

Train (50000, 32, 32, 3) (50000, 1)
Test (10000, 32, 32, 3) (10000, 1)


Load data and scale it from [0,255] to [-1,1]

In [3]:
def load_real_samples():
    (train_x, train_y), (test_x, test_y) = load_data()
    X = np.expand_dims(train_x, axis=-1)
    X = X.astype('float32')
    #X = X / 255.0
    X = (X - 127.5) / 127.5
    return X
    #return [X, train_y]

# Define Models
## Define Discriminator
* Inputs: Image w/3 colour channel and 32x32 pixels
* output: Likelihood whether sample = real or fake
* has a normal convolutional layer followed by three convolutional layers using a stride of 2×2 to downsample the input image
* No pooling layers and a single node in the output layer with the sigmoid activation function to predict whether the input sample is real or fake. 
* The model is trained to minimize the binary cross entropy loss function, appropriate for binary classification.
* use of LeakyReLU instead of ReLU, using Dropout, and using the Adam version of stochastic gradient descent with a learning rate of 0.0002 and a momentum of 0.5.

In [4]:
# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define model
#model = define_discriminator()
# summarize the model
#model.summary()
# plot the model
#plot_model(model, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)


### Sources for real and fake images, generate_real() and generate_fake()
### Generate Real: 
Select a random subsample of images; it will also return class labels for the sample, specifically a class label of 1, to indicate real images

### Generate Fake:
If do not have generator yet, generate images comprised of  random pixel values in the range [0,1], then scaled to the range [-1, 1] like our scaled real images.
Generates images of random pixel values and their associated class label of 0, for fake.

In [5]:
from random import randint, random
from numpy import ones, zeros

# select real samples
def generate_real(dataset, n_samples):
	# choose random instances
	ix = np.random.randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

# generate n fake samples with class labels
"""def generate_fake(n_samples):
	# generate uniform random numbers in [0,1]
	X = np.random.random(32 * 32 * 3 * n_samples)
	# update to have the range [-1, 1]
	X = -1 + X * 2
	# reshape into a batch of color images
	X = X.reshape((n_samples, 32, 32, 3))
	# generate 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y"""
 # use the generator to generate n fake examples, with class labels
def generate_fake(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y


# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = np.random.randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [6]:
#Train discriminator alone

# train the discriminator model
def train_discriminator(model, dataset, n_iter=20, n_batch=128):
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_iter):
		# get randomly selected 'real' samples
		X_real, y_real = generate_real(dataset, half_batch)
		# update discriminator on real samples
		_, real_acc = model.train_on_batch(X_real, y_real)
		# generate 'fake' examples
		X_fake, y_fake = generate_fake(half_batch)
		# update discriminator on fake samples
		_, fake_acc = model.train_on_batch(X_fake, y_fake)
		# summarize performance
		print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))

Discriminator learns how to tell difference betwen  real and randomly generated CIFAR10 images in 20 batches 

## Generator Model
* Responsible for creating new, fake, but plausible small photographs of objects.
* Inputs: Point in latent space, e.g. a 100-element vector of Gaussian random numbers.
* Outputs: Two-dimensional square color image (3 channels) of 32 x 32 pixels with pixel values in [-1,1].
* not trained directly


In [7]:
# define the standalone generator model
def define_generator(latent_dim):
 model = Sequential()
 # foundation for 4x4 image
 n_nodes = 256 * 4 * 4
 model.add(Dense(n_nodes, input_dim=latent_dim))
 model.add(LeakyReLU(alpha=0.2))
 model.add(Reshape((4, 4, 256)))
 # upsample to 8x8
 model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
 model.add(LeakyReLU(alpha=0.2))
 # upsample to 16x16
 model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
 model.add(LeakyReLU(alpha=0.2))
 # upsample to 32x32
 model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
 model.add(LeakyReLU(alpha=0.2))
 # output layer
 model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
 return model
 
# define the size of the latent space
#latent_dim = 100
# define the generator model
#model = define_generator(latent_dim)
# summarize the model
#model.summary()
# plot the model
#plot_model(model, to_file='generator_plot.png', show_shapes=True, show_layer_names=True)

latent space/points = defined under generate fake samples method
Try plotting generated samples

In [8]:
# size of the latent space
from matplotlib import pyplot


#latent_dim = 100
# define the discriminator model
#model = define_generator(latent_dim)
# generate samples
#n_samples = 49
#X, _ = generate_fake(model, latent_dim, n_samples)
# scale pixel values from [-1,1] to [0,1]
#X = (X + 1) / 2.0
# plot the generated samples
#for i in range(n_samples):
 # define subplot
 #pyplot.subplot(7, 7, 1 + i)
 # turn off axis labels
 #pyplot.axis('off')
 # plot single image
 #pyplot.imshow(X[i])
# show the figure
#pyplot.show()

### Train Generator model
* weights in the generator model are updated based on the performance of the discriminator model.
* When the discriminator is good at detecting fake samples, the generator is updated more, and when the discriminator model is relatively poor or confused when detecting fake samples, the generator model is updated less.
* This defines the zero-sum or adversarial relationship between these two models.
* Create a new model that combines the generator and discriminator models. New GAN model can be defined that stacks the generator and discriminator such that the generator receives as input random points in the latent space and generates samples that are fed into the discriminator model directly, classified, and the output of this larger model can be used to update the model weights of the generator.
* logical model that uses the already-defined layers and weights from the standalone generator and discriminator models.

In [9]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# size of the latent space
#latent_dim = 100
# create the discriminator
#d_model = define_discriminator()
# create the generator
#g_model = define_generator(latent_dim)
# create the gan
#gan_model = define_gan(g_model, d_model)
# summarize gan model
#gan_model.summary()
# plot gan model
#plot_model(gan_model, to_file='gan_plot.png', show_shapes=True, show_layer_names=True)

Train both the Discriminator and the Generator with the composite model

In [10]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
     
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			if (i+1) % 10 == 0: summarize_performance(i, g_model, d_model, dataset, latent_dim)
			# get randomly selected 'real' samples
			X_real, y_real = generate_real(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# evaluate the model performance, sometimes

			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
			

In [11]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

In [12]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

In [13]:


# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

2/2 [==============================] - 0s 40ms/step
>1, 1/390, d1=0.682, d2=0.712 g=0.688
2/2 [==============================] - 0s 35ms/step
>1, 2/390, d1=0.389, d2=0.948 g=0.608
2/2 [==============================] - 0s 36ms/step
>1, 3/390, d1=0.139, d2=1.578 g=0.722
2/2 [==============================] - 0s 39ms/step
>1, 4/390, d1=0.183, d2=0.552 g=1.487
2/2 [==============================] - 0s 39ms/step
>1, 5/390, d1=0.225, d2=0.540 g=1.023
2/2 [==============================] - 0s 36ms/step
>1, 6/390, d1=0.240, d2=0.661 g=0.748
2/2 [==============================] - 0s 35ms/step
>1, 7/390, d1=0.158, d2=1.195 g=0.723
2/2 [==============================] - 0s 36ms/step
>1, 8/390, d1=0.122, d2=1.727 g=0.693
2/2 [==============================] - 0s 32ms/step
>1, 9/390, d1=0.292, d2=0.821 g=0.849
2/2 [==============================] - 0s 42ms/step
>1, 10/390, d1=0.545, d2=0.634 g=0.802
2/2 [==============================] - 0s 37ms/step
>1, 11/390, d1=0.179, d2=0.620 g=0.820
2/2 [===